# Jonathan Halverson
# Wednesday, November 29, 2017
# Intro to recurrent neural networks

In [1]:
import tensorflow as tf

Time steps are handled manually.

In [2]:
n_inputs = 3
n_neurons = 5

In [3]:
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

In [4]:
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

In [5]:
Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

In [6]:
init = tf.global_variables_initializer()

In [7]:
import numpy as np

X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) # t=0
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]]) # t=1

In [8]:
with tf.Session() as sess:
     init.run()
     Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0:X0_batch, X1:X1_batch})

In [9]:
print(Y0_val) # output t=0

[[ 0.65553701 -0.93386453 -0.99888057  0.99714911 -0.91205788]
 [-0.36262825 -0.99999964 -1.          0.99999392 -0.81697315]
 [-0.91291749 -1.         -1.          1.         -0.6385175 ]
 [-1.         -0.99999905 -0.12888901 -0.99968666  0.99999964]]


In [10]:
print(Y1_val) # output t=1

[[-0.99939883 -1.         -1.          0.99999171  0.99976784]
 [ 0.53807592 -0.77214199 -0.95488751  0.54583615  0.48453656]
 [-0.98323125 -1.         -1.          0.99792367  0.96890515]
 [-0.88440418 -0.99999481 -0.99080777  0.22809964 -0.86607748]]


### Wednesday, January 3, 2018

Since the output vector at time t is a function of all inputs, the RNN is said to have memory.

In [11]:
import numpy as np
import tensorflow as tf

In [12]:
y_1 = -2.9 # y minus 1
n0 = np.array([1, 2, 3])
n1 = np.array([5, 6, 7])
n2 = np.array([15, -1, 2])

In [13]:
wx = np.array([0.2, 0.7, -0.1])
wy = 0.35
b = 4.12

We use relu here despite tanh being the activation function of choice for RNN's.

In [14]:
def relu(z): return max(0.0, z)

In [15]:
y0 = relu(np.dot(wx, n0) + wy * y_1 + b)
y0

4.4050000000000002

In [16]:
y1 = relu(np.dot(wx, n1) + wy * y0 + b)
y1

10.161749999999998

In [17]:
y2 = relu(np.dot(wx, n2) + wy * y1 + b)
y2

9.7766124999999988

#### Tensorflow implementation

In [18]:
tf.reset_default_graph()

In [19]:
n_inputs = 3
x0 = tf.constant(n0, shape=(1, n_inputs), dtype=tf.float32)
x1 = tf.constant(n1, shape=(1, n_inputs), dtype=tf.float32)
x2 = tf.constant(n2, shape=(1, n_inputs), dtype=tf.float32)

In [20]:
istate = tf.constant([y_1], shape=(1, 1), dtype=tf.float32)

In [21]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=1, activation=tf.nn.relu)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, inputs=[x0, x1, x2], initial_state=istate, dtype=tf.float32)

In [22]:
for v in tf.trainable_variables():
     print(v)

<tf.Variable 'rnn/basic_rnn_cell/weights:0' shape=(4, 1) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/biases:0' shape=(1,) dtype=float32_ref>


In [23]:
graph = tf.get_default_graph()
assign_kernel = graph.get_operation_by_name("rnn/basic_rnn_cell/weights/Assign")
init_kernel = assign_kernel.inputs[1]
assign_bias = graph.get_operation_by_name("rnn/basic_rnn_cell/biases/Assign")
init_bias = assign_bias.inputs[1]

In [24]:
my_w = np.append(wx, wy).reshape(4, 1).astype(np.float32)
my_b = [b]

In [25]:
init = tf.global_variables_initializer()

In [26]:
with tf.Session() as sess:
     sess.run(init, feed_dict={init_kernel:my_w, init_bias:my_b})
     print output_seqs[0].eval()
     print output_seqs[1].eval()
     print output_seqs[2].eval()

[[ 4.40499973]]
[[ 10.16174984]]
[[ 9.77661228]]
